In [2]:
!pip install transformers datasets

# Define the file path
file_path = '/content/zomato.txt'

# Read the content of the file
with open(file_path, 'r') as f:
    text_data = f.read()

print("File content loaded successfully!")
print(text_data[:500])  # Display first 500 characters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
File content loaded successfully!
 e-ISSN: 2582-5208
International Research Journal of Modernization in Engineering Technology and Science
( Peer-Reviewed, Open 

In [3]:
from transformers import GPT2Tokenizer
from datasets import Dataset  # Importing the Dataset class

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Add padding token as GPT-2 does not have a padding token by default
tokenizer.pad_token = tokenizer.eos_token  # Set EOS token as padding token

# Tokenize the text data
encoding = tokenizer(text_data, return_tensors='pt', truncation=True, padding=True, max_length=512)

# Convert into a dataset
train_dataset = Dataset.from_dict({'input_ids': encoding['input_ids'], 'attention_mask': encoding['attention_mask']})

# Check the tokenized dataset format
print(train_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

{'input_ids': [304, 12, 1797, 15571, 25, 1679, 6469, 12, 20, 21315, 198, 24274, 4992, 4913, 286, 12495, 1634, 287, 14044, 8987, 290, 5800, 198, 7, 41139, 12, 40266, 11, 4946, 8798, 11, 40234, 6524, 567, 276, 4037, 4913, 1267, 198, 31715, 25, 3023, 14, 45147, 25, 2931, 14, 17543, 12, 1238, 1828, 17677, 27929, 12, 718, 13, 43665, 7324, 13, 343, 73, 76, 1039, 13, 785, 198, 2503, 13, 343, 73, 76, 1039, 13, 785, 2488, 24274, 4992, 4913, 286, 12495, 1634, 287, 14044, 11, 8987, 290, 5800, 198, 58, 48057, 60, 198, 32, 42001, 49348, 56, 6177, 1168, 2662, 1404, 46, 198, 17822, 24831, 2871, 9, 16, 198, 9, 16, 24095, 1166, 25, 39448, 11, 2059, 5136, 3226, 13473, 11, 20631, 328, 283, 71, 2059, 11, 198, 38, 9869, 7258, 11, 9719, 7344, 11, 33328, 12, 1415, 3023, 1485, 11, 3794, 13, 198, 6242, 18601, 10659, 198, 464, 9793, 286, 4875, 3037, 468, 13105, 262, 8748, 286, 16389, 981, 635, 27179, 9269, 1459, 198, 325, 5217, 13, 383, 3298, 7118, 287, 262, 4875, 6567, 468, 550, 281, 2928, 319, 262, 3942, 3773

In [4]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",           # Output directory
    overwrite_output_dir=True,
    num_train_epochs=2000,                      # Increase number of epochs to improve learning
    per_device_train_batch_size=4,           # Batch size per device
    save_steps=1000,                          # Save checkpoint every 1000 steps
    save_total_limit=2,                      # Keep only 2 checkpoints
    logging_dir="./logs",                    # Directory for logs
    logging_steps=500,                       # Log every 500 steps
    weight_decay=0.01,                       # Weight decay to prevent overfitting
    learning_rate=5e-5,                      # Initial learning rate
    warmup_steps=1000,                       # Number of warmup steps for learning rate scheduler
    lr_scheduler_type="linear",              # Learning rate scheduler type (could be 'linear', 'cosine', etc.)
    gradient_accumulation_steps=8,           # Gradients accumulate over 8 steps before updating the model
    max_grad_norm=1.0,                       # Gradient clipping to avoid exploding gradients
    fp16=True,                               # Use 16-bit precision for faster training (requires a supported GPU)
    evaluation_strategy="no",                # Disable evaluation
    report_to="tensorboard"                  # Use TensorBoard for visualization
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
from transformers import GPT2LMHeadModel
import torch # Import the torch module

# Load the model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize the token embeddings to account for padding
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
from transformers import Trainer
from transformers import DataCollatorForLanguageModeling

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-2 uses causal language modeling (not masked language modeling)
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator

# Start the training process
trainer.train()

# Ensure proper pad token is set
tokenizer.pad_token = "[PAD]"  # Define a new padding token
model.resize_token_embeddings(len(tokenizer))

<ipython-input-6-92239e96fb7f>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.128500
1000,0.000300
1500,0.000100
2000,0.000000


Embedding(50257, 768)

In [7]:
# Input text for generation
input_text = "How case study on Zomato reveals that the company has grown exponentially in the past few years."
inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Move inputs to the model's device
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Generate text with attention mask
generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=100,  # Set a higher max_length for more detailed generation
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.pad_token_id,  # Avoid conflict with EOS
    eos_token_id=tokenizer.eos_token_id
)

# Decode generated output
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


Generated Text: How case study on Zomato reveals that the company has grown exponentially in the past few years. The global expansion in recent years has had an impact on the food sector. With the advent of digital technology, the restaurant sector has embraced an e-commerce infrastructure that allows customers to make orders and have meals delivered to their home. Among many such service providers, Zobato is recognized as one of the most popular applications, offering an online meal delivery service to users who want to discover recipes.

